In [2]:
import os
from os.path import dirname, abspath
import pandas as pd
import valentine as valentine
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import DistributionBased
import pprint

In [3]:
PATH = "C:/Users/fende/Desktop/Dati"
print(PATH)

C:/Users/fende/Desktop/Dati


Load data using pandas

In [4]:
df1 = pd.read_csv(PATH + "/bing_covid-19_data.csv",   sep = ",", low_memory = False)
print(df1.columns)
df2 = pd.read_csv(PATH + "/covid_policy_tracker.csv", sep = ",", low_memory = False)
print(df2.columns)

Index(['id', 'updated', 'confirmed', 'confirmed_change', 'deaths',
       'deaths_change', 'recovered', 'recovered_change', 'latitude',
       'longitude', 'iso2', 'iso3', 'country_region', 'admin_region_1',
       'iso_subdivision', 'admin_region_2', 'load_time'],
      dtype='object')
Index(['countryname', 'countrycode', 'date', 'c1_school_closing', 'c1_flag',
       'c2_workplace_closing', 'c2_flag', 'c3_cancel_public_events', 'c3_flag',
       'c4_restrictions_on_gatherings', 'c4_flag', 'c5_close_public_transport',
       'c5_flag', 'c6_stay_at_home_requirements', 'c6_flag',
       'c7_restrictions_on_internal_movement', 'c7_flag',
       'c8_international_travel_controls', 'e1_income_support', 'e1_flag',
       'e2_debt/contract_relief', 'e3_fiscal_measures',
       'e4_international_support', 'h1_public_information_campaigns',
       'h1_flag', 'h2_testing_policy', 'h3_contact_tracing',
       'h4_emergency_investment_in_healthcare', 'h5_investment_in_vaccines',
       'm1_wildca

Keeping only the common values between the 2 datasets

In [5]:
# presi una tupla a caso per ogni stato della colonna iso3 per evitare di prendere i primi record di ogni stato ed effettuare un random sampling
df1 = df1.groupby(["iso3"]).sample(1)
df2 = df2.groupby(["countrycode"]).sample(1)

merged = pd.merge(df1, df2, how = "inner", left_on='iso3', right_on='countrycode')

print(merged.columns)

col_names_df2 = list(df2.columns)
merged = merged.drop(col_names_df2, axis=1)

merged.to_csv(PATH + "/filtered_bing.csv", index = False)
#df2.to_csv(PATH + "/examples/data/filtered_policy_tracker.csv", index = False)

Index(['id', 'updated', 'confirmed', 'confirmed_change', 'deaths',
       'deaths_change', 'recovered', 'recovered_change', 'latitude',
       'longitude', 'iso2', 'iso3', 'country_region', 'admin_region_1',
       'iso_subdivision', 'admin_region_2', 'load_time', 'countryname',
       'countrycode', 'date', 'c1_school_closing', 'c1_flag',
       'c2_workplace_closing', 'c2_flag', 'c3_cancel_public_events', 'c3_flag',
       'c4_restrictions_on_gatherings', 'c4_flag', 'c5_close_public_transport',
       'c5_flag', 'c6_stay_at_home_requirements', 'c6_flag',
       'c7_restrictions_on_internal_movement', 'c7_flag',
       'c8_international_travel_controls', 'e1_income_support', 'e1_flag',
       'e2_debt/contract_relief', 'e3_fiscal_measures',
       'e4_international_support', 'h1_public_information_campaigns',
       'h1_flag', 'h2_testing_policy', 'h3_contact_tracing',
       'h4_emergency_investment_in_healthcare', 'h5_investment_in_vaccines',
       'm1_wildcard', 'confirmedcases', 

Instantiate matcher and run

In [12]:
%%time
matcher = DistributionBased(0.08, 0.08)
matches = valentine_match(merged, df2, matcher)

CPU times: total: 8.47 s
Wall time: 10.4 s


Ground truth for metrics calculation

In [13]:
ground_truth = [
                ("iso2", "iso_country"),
                ("iso3", "countrycode"),
                ("country_region", "countryname")
               ]

Metrics calculation

In [14]:
metrics = valentine_metrics.all_metrics(matches, ground_truth)

pp = pprint.PrettyPrinter(indent=4)
print("Found the following matches:")
pp.pprint(matches)

print("\nAccording to the ground truth:")
pp.pprint(ground_truth)

print("\nThese are the scores of the matcher:")
pp.pprint(metrics)

Found the following matches:
{   (('table_1', 'confirmed_change'), ('table_2', 'c2_workplace_closing')): 0.9309870418874429,
    (('table_1', 'confirmed_change'), ('table_2', 'h2_testing_policy')): 0.9387057183814522,
    (('table_1', 'country_region'), ('table_2', 'countryname')): 0.9992168591480107,
    (('table_1', 'deaths_change'), ('table_2', 'c1_school_closing')): 0.8499856882178254,
    (('table_1', 'deaths_change'), ('table_2', 'c5_close_public_transport')): 0.9464090944423237,
    (('table_1', 'deaths_change'), ('table_2', 'c6_stay_at_home_requirements')): 0.910639404479407,
    (('table_1', 'deaths_change'), ('table_2', 'c7_restrictions_on_internal_movement')): 0.9559841624603099,
    (('table_1', 'deaths_change'), ('table_2', 'e1_income_support')): 0.8423512051155927,
    (('table_1', 'deaths_change'), ('table_2', 'e2_debt/contract_relief')): 0.8750990667029419,
    (('table_1', 'deaths_change'), ('table_2', 'h3_contact_tracing')): 0.8426424113362708,
    (('table_1', 'iso2'